In [65]:
# Dependencies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import os
import cv2
import random
import pickle

In [66]:
DATADIR = "c:/Users/Rudy/Desktop/DataViz/RoadHazardImageClassification/data"

# Categories for neural network to detect
CATEGORIES = ['cardboard', 'glass', 'metal',
              'paper', 'plastic','potholes', 
              'road_cracks', 'trash']

def get_images(DATADIR):
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
#             plt.imshow(img_array, cmap='gray')
#             plt.show()
#             break
#         break

get_images(DATADIR)

In [67]:
# Print pixel data
print(img_array.shape)
print(img_array)

(384, 512)
[[208 208 209 ... 180 182 183]
 [209 210 211 ... 181 182 182]
 [211 212 212 ... 182 181 181]
 ...
 [155 156 158 ...  95  95  95]
 [155 155 155 ...  94  95  95]
 [157 155 153 ...  94  95  95]]


In [68]:
training_data = []

img_size = 75 # increase based on size of desired item in image

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        for img in os.listdir(path):
            try: 
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

random.shuffle(training_data)

In [69]:
X = [] # features
y = [] # labels

for features, labels in training_data:
    X.append(features)
    y.append(labels)

In [71]:
X = np.array(X).reshape(-1, img_size, img_size, 1)

In [72]:
# Creating the files containing all the information about the model
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

In [81]:
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential, model_from_json, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Dropout

In [74]:
# Data files
X = pickle.load(open('X.pickle', 'rb'))
y = pickle.load(open('y.pickle', 'rb'))

In [75]:
# Normalize dataset
X = X/255 # max number for pixelated data

In [82]:
# Building the model
model = Sequential()
# 3 convolutional layers
model.add(Conv2D(32, (3, 3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))